# Home Assignment No. 2: Part 1 (Theory)

In this part of the homework you are to solve several theoretical problems related to machine learning algorithms.
* For every separate problem you can get only 0 points or maximal points for this problem. There are **NO INTERMEDIATE scores**.
* Your solution must me **COMPLETE**, i.e. contain all required formulas/proofs/detailed explanations.
* You must write your solution for any problem just right after the words **BEGIN SOLUTION**. Attaching pictures of your handwriting is allowed, but **highly discouraged**.
* The are problems with \* mark - they are not obligatory. You can get **EXTRA POINTS** for solving them.
## $\LaTeX$ in Jupyter
Jupyter has constantly improving $\LaTeX$ support. Below are the basic methods to
write **neat, tidy, and well typeset** equations in your notebooks:
* to write an **inline** equation use 
```markdown
$ you latex equation here $
```
* to write an equation, that is **displayed on a separate line** use 
```markdown
$$ you latex equation here $$
```
* to write a **block of equations** use 
```markdown
\begin{align}
    left-hand-side
        &= right-hand-side on line 1
        \\
        &= right-hand-side on line 2
        \\
        &= right-hand-side on the last line
\end{align}
```
The **ampersand** (`&`) aligns the equations horizontally and the **double backslash**
(`\\`) creates a new line.

Write your theoretical derivations within such blocks:
```markdown
**BEGIN Solution**

<!-- >>> your derivation here <<< -->

**END Solution**
```

<br>

## Model and feature selection

### Task 1 (1 pt.): Information criteria

Assume that regression model is
$$y = \sum_{i=1}^k \beta_i x_i + \varepsilon,$$
and $\varepsilon$ is dictributed normally: $\varepsilon \sim \mathcal{N}(0, \sigma^2)$, $\sigma^2$ is known.

Prove that the model with highest Akaike information criterion is the model with smallest Mallow's $C_p$.

**BEGIN Solution**

AIC optimization:

$$
AIC(S) = \ell_J\left(\hat{\beta}\right) - |J| \rightarrow \max\limits_{\hat{\beta}, J}
$$

This problem is equivalent to the following:

$$
-AIC(S) = -\ell_J\left(\hat{\beta}\right) + |J| \rightarrow \min\limits_{\hat{\beta}, J}
$$

Notice that:

$$
\varepsilon_i \mid X_i \sim \mathcal{N}\left(0, \sigma^2\right) \Rightarrow Y_i\mid X_i \sim \mathcal{N}\left(\mu_i, \sigma^2\right),\quad\text{where } \mu_i = \sum\limits_{j \in J}\hat{\beta}_jX_{ij}
$$

Likelihood function:

$$
L_J\left(X, Y \mid \hat{\beta}\right) = \prod\limits_{i = 1}^nf\left(X_i, Y_i \mid \hat{\beta}\right) = \prod\limits_{i = 1}^nf_X\left(X_i \mid \hat{\beta}\right)f_{Y\mid X}\left(Y_i\mid X_i, \hat{\beta}\right)
$$

Logarithmic likelihood function:

$$
\ell_J\left(X, Y \mid \hat{\beta}\right) = \ln L_J\left(X, Y \mid \hat{\beta}\right) = \sum\limits_{i = 1}^n\ln f_X\left(X_i \mid \hat{\beta}\right) + \sum\limits_{i = 1}^n\ln f_{Y\mid X}\left(Y_i\mid X_i, \hat{\beta}\right)
$$

Notice that the first sum doesn't depend on $\hat{\beta}$, therefore we may cancel it out from the optimization problem. Let's take a look at the second one:

$$
\ell_J\left(\hat{\beta}\right) = \sum\limits_{i = 1}^n\ln f_{Y\mid X}\left(Y_i\mid X_i, \hat{\beta}\right) = \sum\limits_{i = 1}^n\ln \left(\frac{1}{\sqrt{2\pi}\sigma}e^{-\frac{(Y_i - \mu_i)^2}{2\sigma^2}}\right) = -n\ln\left(\sqrt{2\pi}\sigma\right) - \frac{1}{2\sigma^2}\sum\limits_{i = 1}^n\left(Y_i - \mu_i\right)^2
$$

Here we can also cancel out the first summand.

Let's get back to the task:

$$
-l_S\left(\hat{\beta}\right) + |J| \rightarrow \min\limits_{\hat{\beta}, J} \sim \frac{1}{2\sigma^2}\sum\limits_{i = 1}^n\left(Y_i - \mu_i\right)^2 + |J| \rightarrow \min\limits_{\hat{\beta}, J} \sim\frac{n}{2\sigma^2}\hat{R}_{tr}\left(J\right) + |J| \rightarrow \min\limits_{\hat{\beta}, J} \sim\hat{R}_{tr}\left(J\right) + \frac{2\sigma^2}{n}|J| \rightarrow \min\limits_{\hat{\beta}, J},
$$

what is indeed a Mallow $C_p$ optimization problem.

Thus, AIC and Mallow $C_p$ optimization problems are equivalent, which means that the model with the highest AIC is the model with the smallest Mallow's $C_p$.

**END Solution**

<br>

## Boosting: gradient boosting, adaboost


### Boosting and its theory

Minimization of the loss function is an optimization task, and "Gradient Boosting"
is one of the many methods to perform optimization. It shoould be noted that it
uses *greedy* approach and therefore, like greedy algorithms in CS, may produce
results that are not *globally* optimal.

$$
\begin{aligned}
    & b_n(x) := \text{the best base model from the family of the algorithms $\mathcal{A}$} \\
    & \gamma_n(x) := \text{scale or weight of the new model} \\
    & a_N(x) = \sum_{n=0}^N \gamma_n b_n(x) := \text{the final composite model}
\end{aligned}
$$

### Gradient Boosting Algorithm

Consider a loss loss function $L(y, z)$ for target $y$ and prediction $z$, and let
$(x_i, y_i)_{i=1}^l$ be our train dataset for a regression task. 


1. Initialize $a_0(x) = \hat{z}$ with the *flat constant prediction*
$\hat{z} = \arg\min\limits_{z \in \mathbb{R}} \sum_{i=1}^l L(y_i, z)$;
2. For $n$ from `1` to `n_boost_steps` do:
    * Solve the current subporblem $G_n(b_n, \gamma_n) \to \min\limits_{b_{n}, \gamma_n}$
    where 
    $$ G_n(b, \gamma) = \sum_{i=1}^l L\bigl(y_i, a_{n-1}(x_i) + \gamma b(x_i)\bigr) $$
    with the following method:
    \begin{align}
      & s_i = - \frac{\partial}{\partial z} L(y_i, z) \Big\vert_{z=a_{n-1}(x_i)}
          \\
      & b_n(x) = \arg\min\limits_{b\in\mathcal{A}}\sum_{i=1}^l \bigl(b(x_i) - s_i\bigr)^2
          \\
      & \gamma_n = \arg\min_\gamma G_n(b_n, \gamma)
          \\
      & a_n(x) = a_{n-1}(x) + \gamma_n b_n(x)
    \end{align}
3. return $a_N(x) = a_0(x) + \sum_{n=1}^N \gamma_n b_n(x)$

<br>

### Task 2 (1 pt.)

At the $n$-th step of Garient Boosting ($n \geq 1$) we compute the "residuals"
$(s_i)_{i=1}^l$ and learn $x\mapsto b_n(x)$ with a regression algorithm $\mathcal{A}$
applied to the dataset $(x_i, s_i)_{i=1}^l$. For the next two tasks **assume
that we have already perfomed these substeps**.

Derive the **optimal value** of $\gamma_n$ for *MSE* loss function $L(y, z) = \tfrac12 (y - z)^2$.

**BEGIN Solution**

$$
G_n\left(b_n, \gamma\right) = \sum\limits_{i=1}^l L\left(y_i, a_{n - 1}(x_i) + \gamma b_n(x_i)\right) = \frac{1}{2}\sum\limits_{i=1}^l\left(y_i - a_{n - 1}(x_i) - \gamma b_n(x_i)\right)^2
$$
$$
\gamma_n = \arg\min_\gamma G_n\left(b_n, \gamma\right)
$$
To find optimal $\gamma_n$, let's make the derivative of $G_n\left(b_n, \gamma\right)$ equal to zero:
$$
\frac{\partial G_n\left(b_n, \gamma\right)}{\partial \gamma} = -\sum\limits_{i=1}^lb_n(x_i)\left(y_i - a_{n-1}(x_i) - \gamma b_n(x_i)\right) = 0 \Rightarrow \gamma\sum\limits_{i=1}^lb_n^2(x_i) = \sum\limits_{i=1}^lb_n(x_i)\left(y_i - a_{n-1}(x_i)\right) \Rightarrow \gamma = \frac{\sum\limits_{i=1}^lb_n(x_i)\left(y_i - a_{n-1}(x_i)\right)}{\sum\limits_{i=1}^lb_n^2(x_i)}
$$
Let's also find the second derivative:
$$
\frac{\partial^2 G_n\left(b_n, \gamma\right)}{\partial \gamma^2} = \sum\limits_{i=1}^lb_n^2(x_i) \geq 0
$$
Therefore, we obtained the optimal value of $\gamma_n$ for MSE loss function:
$$
\gamma_n = \frac{\sum\limits_{i=1}^lb_n(x_i)\left(y_i - a_{n-1}(x_i)\right)}{\sum\limits_{i=1}^lb_n^2(x_i)}
$$


**END Solution**

<br>

### Task 3 (1+1+1 pt.)

Let $S = (x_i, y_i)_{i=1}^l$ be a sample for a classification task $y_i \in \{-1, +1\}$.

The **AdaBoost** algorithm can be regarded as a gradient boosting algorithm
with the exponential loss $L(y,z) = e^{-y z}$. Consider the state of **AdaBoost**
at the $T$-step
$$ G_{T}(b, \gamma)
    = \sum_{i=1}^l L\bigl(y_i, a_{T-1}(x_i) + \gamma b(x_i)\bigr)
    = \sum_{i=1}^l \underbrace{\exp(- y_i a_{T-1}(x_i))}_{w^{T-1}_i}
        \exp(- y_i \gamma b(x_i))
    \,.
$$

#### Task 3.1 (1 pt.)

Derive the next weights $w^T_i$ used in $G_T$ at the stage $T$ as a function
of the learnt classifier $b_T$ and the current weights $w^{T-1}_i$;

#### BEGIN Solution

We know that:
$$
a_T(x_i) = a_{T-1}(x_i) + \gamma_Tb_T(x_i)
$$
Let's multiply two sides by $-y_i$ and take the exponent:
$$
e^{-y_ia_T(x_i)} = e^{-y_ia_{T-1}(x_i)}e^{-y_i\gamma_T b_T(x_i)} \Rightarrow w_i^T = w_i^{T - 1}e^{-y_i\gamma_T b_T(x_i)}
$$
Now let's find $\gamma_T$. Taking into the account that we have a binary classification task with $y_i \in \{-1, 1\}$, we may obtain:
$$
G_T\left(b_T, \gamma\right) = \sum\limits_{i=1}^lw_i^{T-1}e^{-y_i\gamma b_T(x_i)} = \sum\limits_{i=1}^lw_i^{T-1}\left(e^\gamma I\{b_T(x_i) \neq y_i\} + e^{-\gamma}I\{b_T(x_i) = y_i\}\right)
$$
Taking the derivative and making it equal to zero, we get:
$$
\frac{\partial G_T\left(b_T, \gamma\right)}{\partial \gamma} = e^\gamma\sum\limits_{i=1}^lw_i^{T-1}I\{b_T(x_i) \neq y_i\} - e^{-\gamma}\sum\limits_{i=1}^lw_i^{T-1}I\{b_T(x_i) = y_i\} = 0 \Rightarrow e^{2\gamma}\sum\limits_{i=1}^lw_i^{T-1}I\{b_T(x_i) \neq y_i\} = \sum\limits_{i=1}^lw_i^{T-1}I\{b_T(x_i) = y_i\} \Rightarrow \\
\Rightarrow e^{2\gamma} = \frac{\sum\limits_{i=1}^lw_i^{T-1}I\{b_T(x_i) = y_i\}}{\sum\limits_{i=1}^lw_i^{T-1}I\{b_T(x_i) \neq y_i\}} \Rightarrow \gamma = \frac{1}{2}\ln\left(\frac{\sum\limits_{i=1}^lw_i^{T-1}I\{b_T(x_i) = y_i\}}{\sum\limits_{i=1}^lw_i^{T-1}I\{b_T(x_i) \neq y_i\}}\right)
$$
Let's take the second derivative to ensure that this is a point of minimum:
$$
\frac{\partial^2 G_T\left(b_T, \gamma\right)}{\partial \gamma^2} = e^\gamma\sum\limits_{i=1}^lw_i^{T-1}I\{b_T(x_i) \neq y_i\} + e^{-\gamma}\sum\limits_{i=1}^lw_i^{T-1}I\{b_T(x_i) = y_i\} \geq 0,
$$
because $w_i^{T-1}$ are the exponents. Substituting the obtained value of $\gamma_T$ to the expression for $w_i^T$, we can finally get:
$$
w_i^T = w_i^{T - 1}e^{-y_i\gamma_T b_T(x_i)} = w_i^{T - 1}e^{-\frac{y_i}{2}\ln\left(\frac{\sum\limits_{i=1}^lw_i^{T-1}I\{b_T(x_i) = y_i\}}{\sum\limits_{i=1}^lw_i^{T-1}I\{b_T(x_i) \neq y_i\}}\right) b_T(x_i)} = w_i^{T - 1}\left(\frac{\sum\limits_{i=1}^lw_i^{T-1}I\{b_T(x_i) = y_i\}}{\sum\limits_{i=1}^lw_i^{T-1}I\{b_T(x_i) \neq y_i\}}\right)^{-\frac{y_i}{2}b_T(x_i)}
$$

**END Solution**

<br/> <!--Intentionally left blank-->

#### Task 3.2 (1 pt.)

Compute the ratio of weights $(w^T_i)_{i=1}^l$ between the normal and outlier
samples in $S$. Propose a **formal definition of being an outlier**, and reflect
on the value of *margin* for both.

<span style="color:green">**HINT**</span>: *margin* value.

#### BEGIN Solution

Let's take a weight of outlier sample $w_O^T$ and weight of normal sample $w_N^T$ and look at the ratio between them:

$$
\frac{w_O^T}{w_N^T} = \frac{e^{-y_Oa_{T-1}\left(x_O\right)}}{e^{-y_Na_{T-1}\left(x_N\right)}} = e^{y_Na_{T-1}\left(x_N\right) - y_Oa_{T-1}\left(x_O\right)}
$$

Outlier is an object which doesn't match the distribution of the sample. So naturally it should have had another target value, but for some reason (mistake, etc.) it has the one it has. So a classifier may naturally confidently put it in one class when actually it belongs to the other one. This confidence might result in pretty large absolute value of margin $y_Oa_{T-1}\left(x_O\right)$, but with negative sign because prediction and target differ in case of outlier. On the other hand, a good classifier will either put the normal sample in a correct class (probably with big confidence) or make a mistake on it, but won't be sure at all. So, in case of normal sample the margin $y_Na_{T-1}\left(x_N\right)$ will either be positive or small negative. Therefore $e^{y_Na_{T-1}\left(x_N\right) - y_Oa_{T-1}\left(x_O\right)}$ may become relatively large, which means that $w_O^T$ is much bigger than $w_N^T$.

**END Solution**

<br/> <!--Intentionally left blank-->

#### Task 3.3 (1 pt.)

Conclude about **sensitivity** of Adaboost to outliers.

#### BEGIN Solution

As was shown in 3.2, outlier samples have way bigger weights than normal samples. Therefore in case of making a mistake on an outlier, a sample which should ideally have had another target value would cause a large penalty on the classifier, which may significantly change a behaviour of forming a decision boundary, and thus have a large impact on the construction of the final model. It implies that Adaboost is very sensitive to outliers.

**END Solution**

<br/> <!--Intentionally left blank-->

### Task 4 (2+1+2 pt.): Alternative objective functions.

This problem studies boosting-type algorithms defined with objective
functions different from that of AdaBoost.We assume that the training
data are given as m labeled examples $(x_{1},y_{1}),...,(x_{m},y_{m}) \in X \times \{-1,+1\}$.We further assume that $\Phi$ is a strictly increasing convex and differentiable function over $\mathbb{R}$ such that:$\ \forall x \geqslant 0,\Phi(x)\geqslant 1 \ and \ \forall x < 0,\Phi(x) > 0$.

#### Task 4.1 (2 pt.)
Consider the loss function $L(a) =\sum_{i=1}^{m}\Phi \left( -y_{i}g(x_i) \right)$ where $g$ is a linear combination of base classifiers, i.e., $g= \sum_{t=1}^{T} a_t h_t$(as in
AdaBoost). Derive a new boosting algorithm using the objective function $L$. In particular, characterize the best base classifier $h_u$ to select at each round of boosting if we use coordinate descent.

**BEGIN Solution**

Using coordinate descent in the space of algorithms, we can obtain the optimal classifier $h_u$ for the next step:

\begin{eqnarray}
h_u = \arg\min_h\frac{\partial L\left(g + \gamma h\right)}{\partial \gamma}\Big|_{\gamma = 0} =\\
&&= \arg\min_h\frac{\partial \sum\limits_{i=1}^m\Phi\left(-y_i\left(g(x_i) + \gamma h(x_i)\right)\right)}{\partial \gamma}\Big|_{\gamma = 0} =\\
&&= \arg\min_h\left(-\sum\limits_{i=1}^my_ih(x_i)\Phi'\left(-y_ig(x_i)\right)\right) =\\
&&= \left[w_i^T = \Phi'\left(-y_ig(x_i)\right)\right] =\\
&&= \arg\min_h\left(-\sum\limits_{i=1}^mw_i^Ty_ih(x_i)\right) =\\
&&= \arg\min_h\left(-\sum\limits_{i=1}^mw_i^TI\{h(x_i) = y_i\} + \sum\limits_{i=1}^mw_i^TI\{h(x_i) \neq y_i\}\right) =\\
&&= \arg\min_h\sum\limits_{i=1}^mw_i^T\left(I\{h(x_i) \neq y_i\} - I\{h(x_i) = y_i\}\right) =\\
&&= \arg\min_h\sum\limits_{i=1}^mw_i^T\left(2I\{h(x_i) \neq y_i\} - 1\right) =\\
&&= \arg\min_h\sum\limits_{i=1}^mw_i^TI\{h(x_i) \neq y_i\}
\end{eqnarray}

Then let's try to find optimal $\gamma_u$:

\begin{eqnarray}
\gamma_u = \arg\min_\gamma G_T\left(h_u, \gamma\right) =\\
&&= \arg\min_\gamma \sum\limits_{i=1}^m\Phi\left(-y_i\left(g(x_i) + \gamma h_u(x_i)\right)\right) \Rightarrow\\
&&\Rightarrow \left(\sum\limits_{i=1}^m\Phi\left(-y_i\left(g(x_i) + \gamma_u h_u(x_i)\right)\right)\right)' = 0 \Rightarrow\\
&&\Rightarrow \sum\limits_{i=1}^my_ih_u\left(x_i\right)\Phi'\left(y_i\left(g(x_i) + \gamma_uh_u(x_i)\right)\right) = 0
\end{eqnarray}

Therefore, $\gamma_u$ satisfies the equation above and depends on how $\Phi$ looks like.

**END Solution**

<br/> <!--Intentionally left blank-->

#### Task 4.2 (1 pt.)
Consider the following functions:  

1. zero-one loss $\Phi_1(−u) = 1_{u\leqslant0}$;  
2. least squared loss $\Phi_2(−u) = (1 − u)^2$;  
3. SVM loss $\Phi_3(−u) = \max\{0, 1 − u\}$;  
4. logistic loss $\Phi_4(−u) = \log(1 + e^{−u})$.  

Which functions satisfy the assumptions on $\Phi$ stated earlier in this
problem?

Compute the gradient for them.

**BEGIN Solution**

Functions 1-3 doesn't satisfy the assumptions because they are not **strictly** increasing, also it is not true that for $-u < 0$ $\Phi(-u) > 0$. However, function 4 satisfy all the assumptions: it is strictly increasing, for $-u \geq 0$ $\Phi_4(-u) \geq 1$ (assuming $\log$ is $\log_2$) and for $-u < 0$ $\Phi_4(-u) > 0$. Let's find its derivative (gradient):

$$
\frac{\partial \Phi_4(-u)}{\partial(-u)} = \frac{e^{-u}}{\left(1 + e^{-u}\right)\ln2}
$$

It exists everywhere, so $\Phi_4(-u)$ is differentiable. Let's also check that it is convex, taking the second derivative:

$$
\frac{\partial^2 \Phi_4(-u)}{\partial(-u)^2} = \frac{e^{-u}\left(1 + e^{-u}\right) - \left(e^{-u}\right)^2}{\left(1 + e^{-u}\right)^2\ln2} = \frac{e^{-u}}{\left(1 + e^{-u}\right)^2\ln2} \geq 0\qquad\text{for any }u
$$

So, it is convex. Thus, $\Phi_4(-u)$ is the only function among the given four which satisfies all the assumptions. The derivative was computed above.

**END Solution**

<br/> <!--Intentionally left blank-->

#### Task 4.3* (2 pt.)
For each loss function satisfying the assumptions in Task 5 formualtion, derive the
corresponding boosting algorithm. How do the algorithm(s) differ
from AdaBoost?

**BEGIN Solution**

Substituting $\Phi'(-u) = \Phi_4'(-u) = \frac{e^{-u}}{\left(1 + e^{-u}\right)\ln2}$ to the derived equations in Task 4.1, we obtain:

$$
h_u = \arg\min_h\sum\limits_{i=1}^m\Phi'\left(-y_ig(x_i)\right)I\{h(x_i) \neq y_i\} = \arg\min_h\sum\limits_{i=1}^m\frac{e^{-y_ig(x_i)}}{1 + e^{-y_ig(x_i)}}I\{h(x_i) \neq y_i\} = \arg\min_h\sum\limits_{i=1}^m\frac{I\{h(x_i) \neq y_i\}}{1 + e^{y_ig(x_i)}}
$$

Assuming that we obtained $h_u$, then we can get $\gamma_u$:

$$
\begin{eqnarray}
\sum\limits_{i=1}^my_ih_u\left(x_i\right)\Phi'\left(y_i\left(g(x_i) + \gamma_uh_u(x_i)\right)\right) = 0 \Rightarrow\\
&&\Rightarrow \sum\limits_{i=1}^my_ih_u\left(x_i\right)\frac{e^{-y_i\left(g(x_i) + \gamma_uh_u(x_i)\right)}}{\left(1 + e^{-y_i\left(g(x_i) + \gamma_uh_u(x_i)\right)}\right)\ln2} = 0 \Rightarrow\\
&&\Rightarrow \sum\limits_{i=1}^my_ih_u\left(x_i\right)\frac{1}{1 + e^{y_i\left(g(x_i) + \gamma_uh_u(x_i)\right)}} = 0 \Rightarrow\\
&&\Rightarrow \sum\limits_{i=1}^m\left(\frac{I\{h_u(x_i) = y_i\}}{1 + e^{y_ig(x_i) + \gamma_n}} - \frac{I\{h_u(x_i) \neq y_i\}}{1 + e^{y_ig(x_i) - \gamma_n}}\right) = 0\Rightarrow\\
&&\Rightarrow \sum\limits_{i=1}^me^{y_ig(x_i)}\left(e^{\gamma_n}I\{h_u(x_i) \neq y_i\} - e^{-\gamma_n}I\{h_u(x_i) = y_i\}\right) = \sum\limits_{i=1}^m\left(I\{h_u(x_i) = y_i\} - I\{h_u(x_i) \neq y_i\}\right) \Rightarrow\\
&&\Rightarrow e^{2\gamma_n}\sum\limits_{i=1}^me^{y_ig(x_i)}I\{h_u(x_i) \neq y_i\} - e^{\gamma_n}\sum\limits_{i=1}^m\left(I\{h_u(x_i) = y_i\} - I\{h_u(x_i \neq y_i\}\right) - \sum\limits_{i=1}^me^{e^{y_i}g(x_i)}I\{h_u(x_i) = y_i\} = 0,
\end{eqnarray}
$$

which is a quadratic equation on $e^{\gamma_n}$ and can be solved based on the form of $h_u$.

As we can see, the algorithm developed is similar to Adaboost, the only thing which differs is the weights $w_i^T$ because we only changed $\Phi$, and it appears only in them.

**END Solution**

<br/> <!--Intentionally left blank-->

## NNs

### Task 5. (1 pt.)
Consider a two-layer network function of the form
    \begin{equation}
    y_k(x, \mathbf{w}) = \sigma \left ( \sum_{j=1}^M w_{kj}^{(2)} \sigma \left(\sum_{i=1}^D w_{ji}^{(1)}x_i + w_{j0}^{(1)}\right)
                               + w_{k0}^{(2)}\right)
    \end{equation}
in which the nonlinear activation functions are logistic sigmoid functions
    \begin{equation}
    \sigma(a) = (1 + \exp(−a))^{-1}.
    \end{equation}
Show that there exists an equivalent network, which computes exactly the same function but with hidden unit activation function given by hyperbolic tangent ${\rm tanh}(a)$
    \begin{equation}
    {\rm tanh}(a) = \frac{e^a - e^{-a}}{e^a + e^{-a}}.
    \end{equation}


**BEGIN Solution**

Notice that:

$$
{\rm tanh}(a) = \frac{e^a - e^{-a}}{e^a + e^{-a}} = \frac{1 - e^{-2a}}{1 + e^{-2a}} = \left(1 - e^{-2a}\right)\sigma(2a) = \sigma(2a) - \left(1 - \sigma(2a)\right) = 2\sigma(2a) - 1
$$

Therefore:

$$
\sigma(2a) = \frac{{\rm tanh}(a) + 1}{2} \Rightarrow \sigma(a) = \frac{1}{2}{\rm tanh}\left(\frac{a}{2}\right) + \frac{1}{2}
$$

Substituting this to the network function instead of the hidden unit activation function, we obtain:

\begin{eqnarray}
y_k(x, \mathbf{w}) =\\
&&= \sigma\left(\sum_{j=1}^Mw_{kj}^{(2)}\sigma\left(\sum_{i=1}^Dw_{ji}^{(1)}x_i + w_{j0}^{(1)}\right) + w_{k0}^{(2)}\right) =\\
&&= \sigma\left(\sum_{j=1}^M\frac{w_{kj}^{(2)}}{2}{\rm tanh}\left(\sum_{i=1}^D\frac{w_{ji}^{(1)}}{2}x_i + \frac{w_{j0}^{(1)}}{2}\right) + \sum_{j=1}^M\frac{w_{kj}^{(2)}}{2} + w_{k0}^{(2)}\right) =\\
&&= \sigma\left(\sum_{j=1}^Mw_{kj}^{(2)'}{\rm tanh}\left(\sum_{i=1}^Dw_{ji}^{(1)'}x_i + w_{j0}^{(1)'}\right) + w_{k0}^{(2)'}\right),
\end{eqnarray}

where:

$$
w_{j0}^{(1)'} = \frac{1}{2}w_{j0}^{(1)}
$$
$$
w_{ji}^{(1)'} = \frac{1}{2}w_{ji}^{(1)}
$$
$$
w_{k0}^{(2)'} = \sum\limits_{j=1}^M\frac{w_{kj}^{(2)}}{2} + w_{k0}^{(2)}
$$
$$
w_{kj}^{(2)'} = \frac{w_{kj}^{(2)}}{2}
$$

**END Solution**

<br/> <!--Intentionally left blank-->

### Task 6*. Data augmentation (2 pt.)
One way to encourage invariance of a model w.r.t a set of transformations is to expand the training set using transformed versions of the original input patterns.
Consider the framework for training with transformed data in the special case when the transformation is simply addition of random noise $x \rightarrow x + \xi$ where $\xi$ has a Gaussian distribution with zero mean and unit variance. The error function for untransformed inputs can be written (in the infinite data set limit) in the form
    \begin{equation}
    E = \frac12 \int \int (y(\mathbf{x}) - t)^2 p(t|\mathbf{x}) p(\mathbf{x}){\rm d}\mathbf{x} {\rm d}t.
    \end{equation}
If we now consider an infinite number of copies of each data point perturbed by the transformation, then the error function can be written as
    \begin{equation}
    \widetilde{E} = \frac12 \int\int(y(x + \xi) - t)^2p(t | \mathbf{x})p(\mathbf{x}) p(\xi){\rm d}\mathbf{x}{\rm d}t {\rm d}\xi
    \end{equation}
Using Taylor series expansion of $y(\mathbf{x} + \xi)$ show that
    \begin{equation}
    \widetilde{E} \simeq E + \lambda \Omega
    \end{equation}
where $\Omega$ is a regularizer which takes the form of Tikhonov regularizer
    \begin{equation}
    \Omega = \frac12 \int \|\nabla y(\mathbf{x})\|^2 p(\mathbf{x}){\rm d} \mathbf{x}.
    \end{equation}


**BEGIN Solution**

Let's obtain a representation of $y(x + \xi)$ using second-order Taylor series expansion of a multi-variable function:

$$
y(x + \xi) \simeq y(x) + \left(\nabla y(x)\right)^T\xi + \frac{1}{2}\xi^T\left(\nabla^2 y(x)\right)\xi
$$

Subtstituting it to the expression for $\widetilde{E}$, we get:

\begin{eqnarray}
\widetilde{E} \simeq\\
&&\simeq \frac12 \int\int\int\left(y(x) + \left(\nabla y(x)\right)^T\xi + \frac{1}{2}\xi^T\left(\nabla^2 y(x)\right)\xi - t\right)^2p(t | x)p(x) p(\xi){\rm d}x{\rm d}t {\rm d}\xi \simeq\\
&&\simeq \frac12 \int\int\int (y(x) - t)^2 p(t|x) p(x){\rm d}x {\rm d}t + \frac12 \int\int\int\left(\left(\nabla y(x)\right)^T\xi + \frac{1}{2}\xi^T\left(\nabla^2 y(x)\right)\xi\right)^2p(t | x)p(x) p(\xi){\rm d}x{\rm d}t {\rm d}\xi +\\
&&+ \int\int\int(y(x) - t)\left(\left(\nabla y(x)\right)^T\xi + \frac{1}{2}\xi^T\left(\nabla^2 y(x)\right)\xi\right)p(x) p(\xi){\rm d}x{\rm d}t {\rm d}\xi \simeq\\
&&\simeq E + \frac12 \int\int\int\left(\left(\nabla y(x)\right)^T\xi\right)^2p(t | x)p(x) p(\xi){\rm d}x{\rm d}t {\rm d}\xi + \int\int\int(y(x) - t)\left(\left(\nabla y(x)\right)^T\xi + \frac{1}{2}\xi^T\left(\nabla^2 y(x)\right)\xi\right)p(x) p(\xi){\rm d}x{\rm d}t {\rm d}\xi \simeq\\
&&\simeq E + \frac12\int\int\int\left(\left(\nabla y(x)\right)^T\xi\right)^2p(t | x)p(x)p(\xi){\rm d}x{\rm d}t{\rm d}\xi + \int\int(y(x) - t)\left(\nabla y(x)\right)^T\mathbb{E}[\xi] p(t | x)p(x){\rm d}x{\rm d}t{\rm d}\xi +\\
&&+ \frac12\int\int\int(y(x) - t)\xi^T\left(\nabla^2y(x)\right)\xi p(t | x)p(x)p(\xi){\rm d}x{\rm d}t{\rm d}\xi \simeq\\
&&\simeq E + \frac12\int\int\left(\left(\nabla y(x)\right)^T\xi\right)^2p(x)p(\xi){\rm d}x{\rm d}\xi + \frac12\int\int(y(x) - \mathbb{E}\left[t | x\right])\xi^T\left(\nabla^2y(x)\right)\xi p(x)p(\xi){\rm d}x{\rm d}\xi
\end{eqnarray}

Since we are looking for the optimal $y(x)$, we can take a derivative of loss with respect to it and make it equal to zero:

$$
\frac{\partial E}{\partial y(x)} = \int\left(y(x) - t\right)p(t|x)p(x){\rm d}t = 0 \Rightarrow y(x) = \int tp(t|x){\rm d}t = \mathbb{E}\left[t | x\right]
$$

Substituting this to the expression above, we finally obtain:

\begin{eqnarray}
\widetilde{E} \simeq\\
&&\simeq E + \frac12\int\int\left(\left(\nabla y(x)\right)^T\xi\right)^2p(x)p(\xi){\rm d}x{\rm d}\xi\simeq\\
&&\simeq E + \frac12\int\int\xi^T\left(\nabla y(x)\right)\left(\nabla y(x)\right)^T\xi p(x)p(\xi){\rm d}x{\rm d}\xi\simeq\\
&&\simeq E + \frac12\int\int{\rm tr}\left(\left(\left(\nabla y(x)\right)\left(\nabla y(x)\right)^T\right)\left(\xi\xi^T\right)\right) p(x)p(\xi){\rm d}x{\rm d}\xi\simeq\\
&&\simeq E + \frac12\int\int{\rm tr}\left(\left(\nabla y(x)\right)\left(\nabla y(x)\right)^T\right) p(x)p(\xi){\rm d}x{\rm d}\xi\simeq\\
&&\simeq E + \frac12\int\left(\nabla y(x)\right)^T\left(\nabla y(x)\right) p(x){\rm d}x\simeq\\
&&\simeq E + \frac12\int\|\nabla y(x)\|^2 p(x){\rm d}x\simeq\\
&&\simeq E + \Omega,\qquad\text{q.e.d.}
\end{eqnarray}

**END Solution**

<br/> <!--Intentionally left blank-->